# Footprint analytics

Footprint Analytics makes on-chain data available for SQL queries.

The documentation is available at https://docs.footprint.network/docs/get-started

To start a new query, click "Create" and select SQL query.

For example, to query number of unique addresses sending transactions to VVS each week:

```sql
select
concat(
    cast(year(block_timestamp) as VARCHAR),
    '.',
    LPAD(cast(floor(cast(date_format(block_timestamp,'%j') as INT)/7) AS VARCHAR),2,'0')
    ) as time_range,
count(distinct from_address) as unique_wallet
from cronos_transactions
where lower(to_address) = lower('0x145863Eb42Cf62847A6Ca784e6416C1682b1b2Ae') and block_timestamp > timestamp '2022-01-01'
group by 1
order by 1 ASC
```

Note - The query language does not accept aliases of expressions in GROUP BY, use column numbers instead.


# Execute SQL query

The SQL API is only available under the Scale or Enterprise plans.

In [ ]:
import os
import requests
import pprint
from dotenv import load_dotenv  # poetry add python-dotenv
load_dotenv()

footprint_api_key = os.environ.get('FOOTPRINT_API_KEY')

sql = """
select
year(block_timestamp) as tx_year,
week(block_timestamp) as tx_week,
count(distinct from_address) as unique_wallets,
to_address
from cronos_transactions
where lower(to_address) = lower('0x145863Eb42Cf62847A6Ca784e6416C1682b1b2Ae') and block_timestamp > timestamp '2022-01-01'
group by 1, 2, 4
order by 1 ASC, 2 ASC
"""

api_response = requests.request(
    method="post",
    url="https://api.footprint.network/api/v1/native",
    headers={
        "accept": "application/json",
        "content-type": "application/json",
        "api-key": footprint_api_key
    },
    json={
        "query": sql
    }
)
print(api_response.text)
print(api_response.json())